In [7]:
# 参考https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-09-RNN3/
from sklearn.datasets import load_boston
from sklearn import preprocessing
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [40]:
#单值定义
%matplotlib qt
tf.reset_default_graph()
# 波士顿房价数据
boston = load_boston()
x = boston.target
y = boston.target

print('波士顿数据X:', x.shape)  # (506, 1)
# print(x[::100])
print('波士顿房价Y:', y.shape)
# print(y[::100])
# 数据标准化
ss_x = preprocessing.StandardScaler()
train_x = ss_x.fit_transform(y.reshape(-1, 1))
ss_y = preprocessing.StandardScaler()
train_y = ss_y.fit_transform(y.reshape(-1, 1))

BATCH_START = 0  # 建立 batch data 时候的 index
TIME_STEPS = 10  # backpropagation through time 的 time_steps
BATCH_SIZE = 30
INPUT_SIZE = 1  
OUTPUT_SIZE = 1  
CELL_SIZE = 10  # RNN 的 hidden unit size
LR = 0.006  # learning rate
# 保存session状态的位置
save_dir = './bst_save/ckpt/bst'
# logs_path = './bst_save/log/'

# def get_batch_boston_10():
#     global train_x_list, train_y_list, BATCH_START, TIME_STEPS
#     train_x_list,train_y_list=[],[]
#     #(291, 10, 1)
#     for i in range(len(train_y)-TIME_STEPS-205):
#         x1=train_y[i:i+TIME_STEPS]
#         y1=train_y[i+1:i+TIME_STEPS+1]
#     #     x1=x1.reshape(30,10,-1)
#     #     y1=x1.reshape(30,10,-1)
#         train_x_list.append(x1.tolist())
#         train_y_list.append(y1.tolist()) 
#     train_x_np = np.array(train_x_list)
#     train_y_np = np.array(train_y_list)

#     seq = train_x_np
#     res = train_y_np
#     return [seq, res]


# def get_batch_boston_original():
#     global train_x, train_y, BATCH_START, TIME_STEPS
#     # 每次都是拿300行，13列的数据，但是起始点根据 BATCH_START
#     x_part1 = train_x[BATCH_START:BATCH_START + TIME_STEPS * BATCH_SIZE]
#     y_part1 = train_y[BATCH_START:BATCH_START + TIME_STEPS * BATCH_SIZE]
#     # 时间段= 0 300
#     # x_part1= (300, 13)
#     # 时间段= 10 310
#     # x_part1= (300, 13)
# #     print('时间段=', BATCH_START, BATCH_START + TIME_STEPS * BATCH_SIZE)
# #     print('x_part1=', x_part1.shape)
# #     print(BATCH_SIZE, TIME_STEPS, INPUT_SIZE)
#     # 转成 30 10 13的形状
#     seq = x_part1.reshape((BATCH_SIZE, TIME_STEPS, INPUT_SIZE))
#     res = y_part1.reshape((BATCH_SIZE, TIME_STEPS, 1))

#     BATCH_START += TIME_STEPS

#     # returned seq, res and xs: shape (batch, step, input)
#     #np.newaxis 用来增加一个维度 变为三个维度，第三个维度将用来存上一批样本的状态
#     return [seq, res]

#300
def get_batch_boston():
    global train_x_list, train_y_list, BATCH_START, TIME_STEPS
    train_x_list,train_y_list=[],[]
    for i in range(20):
        x1=train_y[i:i+TIME_STEPS*BATCH_SIZE]
        y1=train_y[i+1:i+TIME_STEPS*BATCH_SIZE+1]
#         y1=train_y[i:i+TIME_STEPS*BATCH_SIZE]
        x1=x1.reshape(30,10,-1)
        y1=y1.reshape(30,10,-1)
        train_x_list.append(x1.tolist())
        train_y_list.append(y1.tolist()) 
    train_x_np = np.array(train_x_list)
    train_y_np = np.array(train_y_list)

    seq = train_x_np
    res = train_y_np
    return [seq, res]



# def get_batch():
#     global BATCH_START, TIME_STEPS
#     # xs shape (50batch, 20steps)
#     xs = np.arange(BATCH_START, BATCH_START + TIME_STEPS * BATCH_SIZE).reshape(
#         (BATCH_SIZE, TIME_STEPS)) / (10 * np.pi)
#     print('xs.shape=', xs.shape)
#     seq = np.sin(xs)
#     res = np.cos(xs)
#     BATCH_START += TIME_STEPS
#     # import matplotlib.pyplot as plt
#     # plt.plot(xs[0, :], res[0, :], 'r', xs[0, :], seq[0, :], 'b--')
#     # plt.show()
#     print('增加维度前:', seq.shape)
#     print(seq[:2])
#     print('增加维度后:', seq[:, :, np.newaxis].shape)
#     print(seq[:2])
#     # returned seq, res and xs: shape (batch, step, input)
#     #np.newaxis 用来增加一个维度 变为三个维度，第三个维度将用来存上一批样本的状态
#     return [seq[:, :, np.newaxis], res[:, :, np.newaxis], xs]


class LSTMRNN(object):
    def __init__(self, n_steps, input_size, output_size, cell_size,
                 batch_size):
        '''
        :param n_steps: 每批数据总包含多少时间刻度
        :param input_size: 输入数据的维度
        :param output_size: 输出数据的维度 如果是类似价格曲线的话，应该为1
        :param cell_size: cell的大小
        :param batch_size: 每批次训练数据的数量
        '''
        self.n_steps = n_steps
        self.input_size = input_size
        self.output_size = output_size
        self.cell_size = cell_size
        self.batch_size = batch_size
        with tf.name_scope('inputs'):
            self.xs = tf.placeholder(
                tf.float32, [None, n_steps, input_size], name='xs')  #xs 有三个维度
            self.ys = tf.placeholder(
                tf.float32, [None, n_steps, output_size], name='ys')  #ys 有三个维度
        with tf.variable_scope('in_hidden'):
            self.add_input_layer()
        with tf.variable_scope('LSTM_cell'):
            self.add_cell()
        with tf.variable_scope('out_hidden'):
            self.add_output_layer()
        with tf.name_scope('cost'):
            self.compute_cost()
        with tf.name_scope('train'):
            self.train_op = tf.train.AdamOptimizer(LR).minimize(self.cost)

    #增加一个输入层
    def add_input_layer(self, ):
        # l_in_x:(batch*n_step, in_size),相当于把这个批次的样本串到一个长度1000的时间线上，每批次50个样本，每个样本20个时刻
        l_in_x = tf.reshape(
            self.xs, [-1, self.input_size], name='2_2D')  #-1 表示任意行数
        # Ws (in_size, cell_size)
        Ws_in = self._weight_variable([self.input_size, self.cell_size])
        # bs (cell_size, )
        bs_in = self._bias_variable([
            self.cell_size,
        ])
        # l_in_y = (batch * n_steps, cell_size)
        with tf.name_scope('Wx_plus_b'):
            l_in_y = tf.matmul(l_in_x, Ws_in) + bs_in
        # reshape l_in_y ==> (batch, n_steps, cell_size)
        self.l_in_y = tf.reshape(
            l_in_y, [-1, self.n_steps, self.cell_size], name='2_3D')

    #多时刻的状态叠加层 CELL
    def add_cell(self):
#         print('BATCH_SIZE',self.batch_size)
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(
            self.cell_size, forget_bias=1.0, state_is_tuple=True)
        with tf.name_scope('initial_state'):
            self.cell_init_state = lstm_cell.zero_state(
                self.batch_size, dtype=tf.float32)
        #time_major=False 表示时间主线不是第一列batch
        self.cell_outputs, self.cell_final_state = tf.nn.dynamic_rnn(
            lstm_cell,
            self.l_in_y,
            initial_state=self.cell_init_state,
            time_major=False)

    # 增加一个输出层
    def add_output_layer(self):
        # shape = (batch * steps, cell_size)
        l_out_x = tf.reshape(
            self.cell_outputs, [-1, self.cell_size], name='2_2D')
        Ws_out = self._weight_variable([self.cell_size, self.output_size])
        bs_out = self._bias_variable([
            self.output_size,
        ])
        # shape = (batch * steps, output_size)
        with tf.name_scope('Wx_plus_b'):
            self.pred = tf.matmul(l_out_x, Ws_out) + bs_out  #预测结果
    
    # 计算损失
    def compute_cost(self):
        losses = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [tf.reshape(self.pred, [-1], name='reshape_pred')],
            [tf.reshape(self.ys, [-1], name='reshape_target')],
            [tf.ones([self.batch_size * self.n_steps], dtype=tf.float32)],
            average_across_timesteps=True,
            softmax_loss_function=self.ms_error,
            name='losses')
        with tf.name_scope('average_cost'):
            self.cost = tf.div(
                tf.reduce_sum(losses, name='losses_sum'),
                self.batch_size,
                name='average_cost')
            tf.summary.scalar('cost', self.cost)

    def ms_error(self, labels, logits):
        return tf.square(tf.subtract(labels, logits))

    def _weight_variable(self, shape, name='weights'):
        initializer = tf.random_normal_initializer(
            mean=0.,
            stddev=1.,
        )
        return tf.get_variable(shape=shape, initializer=initializer, name=name)

    def _bias_variable(self, shape, name='biases'):
        initializer = tf.constant_initializer(0.1)
        return tf.get_variable(name=name, shape=shape, initializer=initializer)



波士顿数据X: (506,)
波士顿房价Y: (506,)


In [41]:
# 训练
tf.reset_default_graph()
if __name__ == '__main__':
    # 设置画布初始属性和内容
    
    fig = plt.figure(figsize=(20, 3))  # dpi参数指定绘图对象的分辨率，即每英寸多少个像素，缺省值为80
    axes = fig.add_subplot(1, 1, 1)
    line3, = axes.plot(train_y[19:-1], 'r', label='实际')
    plt.ion()
    plt.show()
    
    seq, res = get_batch_boston()
    costlist = []
    print('seq',seq.shape)
    print('res',res.shape)
    BATCH_SIZE  = 30
    model = LSTMRNN(TIME_STEPS, INPUT_SIZE, OUTPUT_SIZE, CELL_SIZE, BATCH_SIZE)
    sess = tf.Session()
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("logs", sess.graph)
    # tf.initialize_all_variables() no long valid from
    # 2017-03-02 if using tensorflow >= 0.12
    sess.run(tf.global_variables_initializer())
    # relocate to the local dir and run this line to view it on Chrome (http://0.0.0.0:6006/):
    # $ tensorboard --logdir='logs'
    for j in range(200):  #训练200次
        pred_res = None
        # 300
        seq, res = get_batch_boston()
#         print('seq',seq.shape)
#        用于300
        for i in range(20):
            if i == 0:
#                 print('if')
                feed_dict = {
                    model.xs: seq[i],
                    model.ys: res[i],
                    # create initial state
                }
            else:
#                 print('else')
                feed_dict = {
                    model.xs: seq[i],
                    model.ys: res[i],
                    model.cell_init_state:
                    state  # use last state as the initial state for this run
                }
#             print('se',seq[start:end].shape)
#             start += TIME_STEPS
#             print('start',type(start))
#             end=start+BATCH_SIZE

#         10
#         step=0
#         start=0
#         end=start+BATCH_SIZE
#         #每个内循环是100次，数据是0-60，60-120，120-180，递增
#         while(end<len(train_x)-206):
#             print()
#             if start == 0:
# #                 print('if')
#                 feed_dict = {
#                     model.xs: seq[start:end],
#                     model.ys: res[start:end],
#                     # create initial state
#                 }
#             else:
# #                 print('else')
#                 feed_dict = {
#                     model.xs: seq[start:end],
#                     model.ys: res[start:end],
#                     model.cell_init_state:
#                     state  # use last state as the initial state for this run
#                 }
#             print('start,end,step',start,end,step,',seq',seq[start:end].shape)
#             start+=BATCH_SIZE
#             end=start+BATCH_SIZE
#             #每10步保存一次参数
#             if step%10==0:
#                 print(i,step,cost)
#             step+=1

            
            _, cost, state, pred = sess.run([
                model.train_op, model.cost, model.cell_final_state, model.pred
            ],feed_dict=feed_dict)
            pred_res = pred
            


            result = sess.run(merged, feed_dict)
            writer.add_summary(result, i)
        if(j % 10 ==0):
            print('{0} cost: '.format(j), round(cost, 4))
        BATCH_START = 0  #从头再来一遍
        try:
            axes.lines.remove(line1)
            axes.lines.remove(line_c)
        except Exception:
            pass
        
        costlist.append(cost)
        line_c, = axes.plot(costlist,'y',label='cost')
#         handles, labels = plt.gca().get_legend_handles_labels()
        # 返回是, 是一个Line2D的对象，无逗号是一个包含了Line2D的数组
#         line1, = axes.plot(
#             range(100), pred.flatten()[-100:], 'b--', label='rnn计算结果 '+str(j))
        line1, = axes.plot(pred.flatten(), 'b--', label='rnn计算结果 '+str(j))
        axes.grid()
        fig.tight_layout()
        plt.legend(handles=[line1,line3])
        plt.title('递归神经网络')
        plt.pause(0.01)
    plt.ioff()
    plt.show()
    
    # 保存模型
#     saver = tf.train.Saver()
#     saver.save(sess, save_dir,)
    print('Model Trained and Saved')

seq (20, 30, 10, 1)
res (20, 30, 10, 1)
0 cost:  4.9987
10 cost:  3.8869
20 cost:  3.9777
30 cost:  4.0459
40 cost:  3.9228
50 cost:  3.189
60 cost:  3.7168
70 cost:  3.3718
80 cost:  3.2141
90 cost:  2.9967
100 cost:  2.7817
110 cost:  3.1389
120 cost:  2.3358
130 cost:  2.519
140 cost:  2.3486
150 cost:  2.12
160 cost:  2.5229
170 cost:  1.9962
180 cost:  2.4313
190 cost:  2.4486
Model Trained and Saved


In [48]:
# 预测 300 可用
%matplotlib qt
gen_length = 500
prime_word = '1'
loaded_graph = tf.Graph()
tf.reset_default_graph()
seq, res = get_batch_boston()
# BATCH_SIZE  = 1 
#改变 cell 的尺寸 和 输入数据的尺寸一致
model = LSTMRNN(TIME_STEPS, INPUT_SIZE, OUTPUT_SIZE, CELL_SIZE, 1)
saver = tf.train.Saver()
fig = plt.figure(figsize=(20, 3))  # dpi参数指定绘图对象的分辨率，即每英寸多少个像素，缺省值为80
axes = fig.add_subplot(1, 1, 1)
# train_y1 = train_y[190:490]
# train_y2 = train_y[0:300]
# train_y3 = train_y[0:506]
line5, = axes.plot(train_y.flatten(), 'y', label='实际5')
plt.ion()
plt.show()
print('seq', seq.shape)
with tf.Session() as sess:
    # 加载保存过的session
    #     loader = tf.train.import_meta_graph(save_dir + '.meta')
    saver.restore(sess, './bst_save/ckpt/bst')
    #取训练集的最后一行，作为开始
#     prev_seq=train_x[-1]
#     prev_seq = seq[-1]
    prev_seq = np.array([seq.reshape(-1,10,1)[0].tolist()])
#      可用
#     prev_seq = np.array([train_x_np[50].tolist()])
    print('prev_seq', prev_seq.shape)
    predict=[]
    for i in range(200):
#         print('prev_seq_in',prev_seq)
        if i == 0:
#             print('if')
            feed_dict = {
                # create initial state
                model.xs: prev_seq
            }
        else:
#             print('else')
            feed_dict = {
                model.xs: prev_seq,
                # use last state as the initial state for this run
                model.cell_init_state: state 
            }
        state, next_seq = sess.run([model.cell_final_state, model.pred],
                               feed_dict=feed_dict)
#         print('next_seq',next_seq.shape)
#         print('next_seq',next_seq[-1])
#         print(type(predict))
#       下次的训练数据都是老一代的1-X加预测值的最后一位，所以数据可视化的时候，只需要展出最后更新那一位就可以了
#       梯度消失
        predict.append(next_seq[-1])
#         predict = np.append(predict,next_seq)
#         print('prev_seq',prev_seq.reshape([-1]).shape
#         print('prev_seq',prev_seq.shape)
#         print('next_seq',next_seq.shape)
#         print('next_seq',next_seq[-1].shape)
        print('prev_seq_out',prev_seq)
#        丢弃首位，插入末位，输入的数据保持0-8位的一致性，形成连续
#         prev_seq = np.append(prev_seq.reshape(TIME_STEPS)[1:],next_seq.reshape(TIME_STEPS)[-1]).reshape(1,10,1)
#         prev_seq = prev_seq.reshape(1,10,1)

#         print('prev_seq[1:]',prev_seq[1:])
#        全新输入，梯度爆炸
        prev_seq = [next_seq]
#         prev_seq = np.vstack((prev_seq[1:],[next_seq]))

        line1, = axes.plot(predict, 'b--', label='rnn计算结果' + str(i))
        axes.grid()
        fig.tight_layout()
        plt.legend(handles=[line1, line5])
        plt.title('递归神经网络')
        plt.pause(0.01)

    BATCH_START = 0
    plt.ioff()
    plt.show()

seq (20, 30, 10, 1)
INFO:tensorflow:Restoring parameters from ./bst_save/ckpt/bst
prev_seq (1, 10, 1)
prev_seq_out [[[ 0.15968566]
  [-0.10152429]
  [ 1.32424667]
  [ 1.18275795]
  [ 1.48750288]
  [ 0.6712218 ]
  [ 0.03996443]
  [ 0.49708184]
  [-0.65659542]
  [-0.39538548]]]
prev_seq_out [array([[ 0.16351035],
       [-0.06368324],
       [ 1.2669452 ],
       [ 1.2167677 ],
       [ 1.5328723 ],
       [ 0.7166994 ],
       [ 0.07408087],
       [ 0.4918877 ],
       [-0.678943  ],
       [-0.43444115]], dtype=float32)]
prev_seq_out [array([[ 0.12413   ],
       [-0.0726745 ],
       [ 1.284955  ],
       [ 1.2414328 ],
       [ 1.5586842 ],
       [ 0.72109014],
       [ 0.07697739],
       [ 0.48038816],
       [-0.70061356],
       [-0.45714402]], dtype=float32)]
prev_seq_out [array([[ 0.10953857],
       [-0.06753837],
       [ 1.3210515 ],
       [ 1.2737321 ],
       [ 1.588334  ],
       [ 0.7252367 ],
       [ 0.07873495],
       [ 0.47006458],
       [-0.7206623 ],
       [-

prev_seq_out [array([[ 0.22467043],
       [ 0.60621005],
       [ 2.9080317 ],
       [ 2.930147  ],
       [ 2.9044247 ],
       [ 1.0841221 ],
       [ 0.16816075],
       [ 0.5176633 ],
       [-1.0462208 ],
       [-1.1270001 ]], dtype=float32)]
prev_seq_out [array([[ 0.24227615],
       [ 0.67091197],
       [ 2.9255743 ],
       [ 2.9358263 ],
       [ 2.9132724 ],
       [ 1.1080099 ],
       [ 0.18785079],
       [ 0.5326264 ],
       [-1.0468184 ],
       [-1.1364442 ]], dtype=float32)]
prev_seq_out [array([[ 0.25953072],
       [ 0.74179107],
       [ 2.9426796 ],
       [ 2.9404917 ],
       [ 2.9210968 ],
       [ 1.1335195 ],
       [ 0.21064062],
       [ 0.5486662 ],
       [-1.0471069 ],
       [-1.1460261 ]], dtype=float32)]
prev_seq_out [array([[ 0.27640933],
       [ 0.8199845 ],
       [ 2.9593306 ],
       [ 2.9443922 ],
       [ 2.9281156 ],
       [ 1.1607509 ],
       [ 0.23673435],
       [ 0.5658765 ],
       [-1.0470163 ],
       [-1.1558297 ]], dtype=float3

prev_seq_out [array([[ 1.9275147 ],
       [ 3.2289877 ],
       [ 3.006031  ],
       [ 2.9897895 ],
       [ 3.010827  ],
       [ 2.8670547 ],
       [ 1.7931889 ],
       [ 1.8520933 ],
       [-0.44684464],
       [-1.131612  ]], dtype=float32)]
prev_seq_out [array([[ 2.0563788 ],
       [ 3.2276294 ],
       [ 3.003958  ],
       [ 2.9911857 ],
       [ 3.0112953 ],
       [ 2.8970146 ],
       [ 1.8586636 ],
       [ 1.9281616 ],
       [-0.40942276],
       [-1.111321  ]], dtype=float32)]
prev_seq_out [array([[ 2.185272  ],
       [ 3.224747  ],
       [ 3.0019352 ],
       [ 2.9924529 ],
       [ 3.0116372 ],
       [ 2.9217672 ],
       [ 1.9265045 ],
       [ 2.0076754 ],
       [-0.37074405],
       [-1.0907809 ]], dtype=float32)]
prev_seq_out [array([[ 2.3110473],
       [ 3.220521 ],
       [ 2.9999895],
       [ 2.9935942],
       [ 3.0118642],
       [ 2.9419727],
       [ 1.9967258],
       [ 2.090335 ],
       [-0.3306669],
       [-1.070333 ]], dtype=float32)]
prev_s

prev_seq_out [array([[ 3.0254862 ],
       [ 3.0914788 ],
       [ 2.9962308 ],
       [ 3.015089  ],
       [ 3.023238  ],
       [ 3.0311744 ],
       [ 3.0366206 ],
       [ 3.0455177 ],
       [ 1.4654322 ],
       [-0.44552487]], dtype=float32)]
prev_seq_out [array([[ 3.0273893 ],
       [ 3.087544  ],
       [ 2.9969792 ],
       [ 3.0160916 ],
       [ 3.0241795 ],
       [ 3.032081  ],
       [ 3.037972  ],
       [ 3.0464213 ],
       [ 1.5317774 ],
       [-0.39614677]], dtype=float32)]
prev_seq_out [array([[ 3.0295105 ],
       [ 3.0834317 ],
       [ 2.997788  ],
       [ 3.0171537 ],
       [ 3.0251806 ],
       [ 3.033043  ],
       [ 3.039263  ],
       [ 3.0473442 ],
       [ 1.600848  ],
       [-0.34238565]], dtype=float32)]
prev_seq_out [array([[ 3.031884  ],
       [ 3.079155  ],
       [ 2.9986715 ],
       [ 3.0182874 ],
       [ 3.0262518 ],
       [ 3.0340703 ],
       [ 3.0405297 ],
       [ 3.048304  ],
       [ 1.6730288 ],
       [-0.28364265]], dtype=float3

prev_seq_out [array([[3.1587775],
       [3.140199 ],
       [3.1419601],
       [3.142866 ],
       [3.14296  ],
       [3.143245 ],
       [3.1434848],
       [3.1437075],
       [3.1432993],
       [3.083348 ]], dtype=float32)]
prev_seq_out [array([[3.1565864],
       [3.1416826],
       [3.1432085],
       [3.1438951],
       [3.1439536],
       [3.1441727],
       [3.1443543],
       [3.144524 ],
       [3.1442435],
       [3.100999 ]], dtype=float32)]
prev_seq_out [array([[3.1547184],
       [3.1428742],
       [3.1441824],
       [3.1446972],
       [3.1447313],
       [3.144899 ],
       [3.1450357],
       [3.1451638],
       [3.1449716],
       [3.1139104]], dtype=float32)]
prev_seq_out [array([[3.153161 ],
       [3.1438258],
       [3.1449347],
       [3.1453173],
       [3.1453345],
       [3.1454625],
       [3.145565 ],
       [3.1456614],
       [3.145529 ],
       [3.123289 ]], dtype=float32)]
prev_seq_out [array([[3.1518853],
       [3.1445804],
       [3.1455116],
  

prev_seq_out [array([[3.147241 ],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406]], dtype=float32)]
prev_seq_out [array([[3.1472409],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406]], dtype=float32)]
prev_seq_out [array([[3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406]], dtype=float32)]
prev_seq_out [array([[3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406],
       [3.1472406]], dtype=float32)]
prev_seq_out [array([[3.1472406],
       [3.1472406],
       [3.1472406],
  

In [ ]:
# 预测 10
%matplotlib qt
gen_length = 500
prime_word = '1'
loaded_graph = tf.Graph()
tf.reset_default_graph()
seq, res = get_batch_boston()
# BATCH_SIZE  = 1 
#改变 cell 的尺寸 和 输入数据的尺寸一致
model = LSTMRNN(TIME_STEPS, INPUT_SIZE, OUTPUT_SIZE, CELL_SIZE, 1)
saver = tf.train.Saver()
fig = plt.figure(figsize=(20, 3))  # dpi参数指定绘图对象的分辨率，即每英寸多少个像素，缺省值为80
axes = fig.add_subplot(1, 1, 1)
# train_y1 = train_y[190:490]
# train_y2 = train_y[0:300]
# train_y3 = train_y[0:506]
line5, = axes.plot(train_y.flatten(), 'y', label='实际5')
plt.ion()
plt.show()
print('seq', seq.shape)
with tf.Session() as sess:
    # 加载保存过的session
    #     loader = tf.train.import_meta_graph(save_dir + '.meta')
    saver.restore(sess, './bst_save/ckpt/bst')
    #取训练集的最后一行，作为开始
#     prev_seq=train_x[-1]
#     prev_seq = seq[-1]
    prev_seq = np.array([train_x_np[0][0].tolist()])
    print('prev_seq', prev_seq.shape)
    predict=[]
    for i in range(20):
        if i == 0:
#             print('if')
            feed_dict = {
                # create initial state
                model.xs: prev_seq
            }
        else:
#             print('else')
            feed_dict = {
                model.xs: prev_seq,
                # use last state as the initial state for this run
                model.cell_init_state: state 
            }
        state, next_seq = sess.run([model.cell_final_state, model.pred],
                               feed_dict=feed_dict)
#         print('next_seq',next_seq.shape)
#         print('next_seq',next_seq[-1])
#         print(type(predict))
        predict.append(next_seq[-1])
        
#         print('prev_seq',prev_seq.reshape([-1]).shape
#         print('prev_seq',prev_seq.shape)
#         print('next_seq',next_seq.shape)
#         print('next_seq',next_seq[-1].shape)
        prev_seq = np.vstack((prev_seq[1:],[next_seq]))

        line1, = axes.plot(predict, 'b--', label='rnn计算结果' + str(i))
        axes.grid()
        fig.tight_layout()
        plt.legend(handles=[line1, line5])
        plt.title('递归神经网络')
        plt.pause(0.5)

    BATCH_START = 0
    plt.ioff()
    plt.show()

# 

# 

In [30]:
# %matplotlib qt
gen_length = 500
prime_word = '1'
loaded_graph = tf.Graph()
tf.reset_default_graph()
seq, res = get_batch_boston()
model = LSTMRNN(TIME_STEPS, INPUT_SIZE, OUTPUT_SIZE, CELL_SIZE, BATCH_SIZE)
saver = tf.train.Saver()
fig = plt.figure(figsize=(20, 3))  # dpi参数指定绘图对象的分辨率，即每英寸多少个像素，缺省值为80
axes = fig.add_subplot(1, 1, 1)
train_y1 = train_y[190:490]
train_y2 = train_y[0:300]
train_y3 = train_y
# train_y1 = train_y[190:300]
# train_y2 = train_y[190:300]

print(type(train_y1))
print(train_y1.shape)
# train_y1.flatten()[-100:]
# line3, = axes.plot(train_y1.flatten(), 'r', label='实际1>190')

line5, = axes.plot(train_y3.flatten(), 'y', label='实际3')
line4, = axes.plot(train_y2.flatten(), 'g', label='实际2>0')
plt.legend(handles=[line4,line3])
plt.ion()
plt.show()

时间段= 170 470
x_part1= (300, 13)
<class 'numpy.ndarray'>
(300, 1)


In [261]:
seq, res = get_batch_boston()
print(seq.shape)
print(train_x_np.shape)
# print(seq.reshape(-1,10,1)[0])
#0-29：0-290：0：10
#30-59：1-291，31：11，32：21
print(seq.reshape(-1,10,1)[50])
print()
# print(train_x_np[0])
print(train_x_np[201])

(20, 30, 10, 1)
(291, 10, 1)
[[ 0.17056941]
 [ 2.1514115 ]
 [ 2.82620386]
 [ 2.98946007]
 [ 0.00731319]
 [ 0.20322065]
 [-0.00357056]
 [ 0.20322065]
 [-0.27566425]
 [-0.09064054]]

[[ 0.17056941]
 [ 2.1514115 ]
 [ 2.82620386]
 [ 2.98946007]
 [ 0.00731319]
 [ 0.20322065]
 [-0.00357056]
 [ 0.20322065]
 [-0.27566425]
 [-0.09064054]]


In [131]:
a=np.arange(6).reshape(3,2)
print(a)
print(a.flatten())
print(a.flatten()[-2:])

[[0 1]
 [2 3]
 [4 5]]
[0 1 2 3 4 5]
[4 5]


In [32]:
plt.figure()

plt.subplot(2,1,1)      # 分成两行一列，起始点为1
plt.plot([0,1],[0,1])   # 设置xy轴范围

plt.subplot(2,3,4)      # 分成两行三列，起始点位4
plt.plot([0,1],[0,2])

plt.subplot(2,3,5)
plt.plot([0,1],[0,3])

plt.subplot(2,3,6)
plt.plot([0,1],[0,4])

plt.show()

In [10]:
a=np.arange(56).reshape(8,7)
#X取前5行的前三列（从1计数）
x=a[0:0+5,:3]
#Y取前5行的第4列（从0计数），并行转列
y=a[0:0+5,0,np.newaxis]
z=a[:,0]
print(a)
print(z)
zz =  (a[:,0] * 1000 / 60) - (a[:,4] + a[:,5] + a[:,6])
print(zz)
print('a',a[1:])
# print(x.shape)
# print(x)
# print(y.shape)
# print(y)

[[ 0  1  2  3  4  5  6]
 [ 7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27]
 [28 29 30 31 32 33 34]
 [35 36 37 38 39 40 41]
 [42 43 44 45 46 47 48]
 [49 50 51 52 53 54 55]]
[ 0  7 14 21 28 35 42 49]
[-15.          80.66666667 176.33333333 272.         367.66666667
 463.33333333 559.         654.66666667]
a [[ 7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27]
 [28 29 30 31 32 33 34]
 [35 36 37 38 39 40 41]
 [42 43 44 45 46 47 48]
 [49 50 51 52 53 54 55]]


In [137]:
train_x_list,train_y_list=[],[]
for i in range(20):
    x1=train_y[i:i+TIME_STEPS*BATCH_SIZE]
    y1=train_y[i+1:i+TIME_STEPS*BATCH_SIZE+1]
    x1=x1.reshape(30,10,-1)
    y1=x1.reshape(30,10,-1)
    train_x_list.append(x1.tolist())
    train_y_list.append(y1.tolist()) 
    print(i,i+TIME_STEPS*BATCH_SIZE)
print(len(train_x_list))
print(len(train_y_list))
train_x_np = np.array(train_x_list)
train_y_np = np.array(train_y_list)

0 300
1 301
2 302
3 303
4 304
5 305
6 306
7 307
8 308
9 309
10 310
11 311
12 312
13 313
14 314
15 315
16 316
17 317
18 318
19 319
20
20


In [192]:
train_x_list,train_y_list=[],[]
for i in range(len(train_y)-TIME_STEPS-205):
    x1=train_y[i:i+TIME_STEPS]
    y1=train_y[i+1:i+TIME_STEPS+1]
#     x1=x1.reshape(30,10,-1)
#     y1=x1.reshape(30,10,-1)
    train_x_list.append(x1.tolist())
    train_y_list.append(y1.tolist()) 
    print(i,i+TIME_STEPS)
# print(len(train_x_list))
# print(len(train_y_list))
train_x_np = np.array(train_x_list)
train_y_np = np.array(train_y_list)

0 10
1 11
2 12
3 13
4 14
5 15
6 16
7 17
8 18
9 19
10 20
11 21
12 22
13 23
14 24
15 25
16 26
17 27
18 28
19 29
20 30
21 31
22 32
23 33
24 34
25 35
26 36
27 37
28 38
29 39
30 40
31 41
32 42
33 43
34 44
35 45
36 46
37 47
38 48
39 49
40 50
41 51
42 52
43 53
44 54
45 55
46 56
47 57
48 58
49 59
50 60
51 61
52 62
53 63
54 64
55 65
56 66
57 67
58 68
59 69
60 70
61 71
62 72
63 73
64 74
65 75
66 76
67 77
68 78
69 79
70 80
71 81
72 82
73 83
74 84
75 85
76 86
77 87
78 88
79 89
80 90
81 91
82 92
83 93
84 94
85 95
86 96
87 97
88 98
89 99
90 100
91 101
92 102
93 103
94 104
95 105
96 106
97 107
98 108
99 109
100 110
101 111
102 112
103 113
104 114
105 115
106 116
107 117
108 118
109 119
110 120
111 121
112 122
113 123
114 124
115 125
116 126
117 127
118 128
119 129
120 130
121 131
122 132
123 133
124 134
125 135
126 136
127 137
128 138
129 139
130 140
131 141
132 142
133 143
134 144
135 145
136 146
137 147
138 148
139 149
140 150
141 151
142 152
143 153
144 154
145 155
146 156
147 157
148 158
149 159


In [208]:
# print(train_x_np[0].shape)
# print(train_x_np[0][0].shape)
print(train_x_np.shape)
print(train_x_np[270:300].shape)
# print('a',train_x_np[0])
print('b',type(train_x_np[np.newaxis][0][0]))
[train_x_np[0][0].tolist()]

print(np.array([train_x_np[0][0].tolist()]))
print('len',len(train_x)-205)
print(train_x_np.shape)
print(train_y_np.shape)
# print(y.shape)
# print(train_y.shape)
seq, res = get_batch_boston()
print('seq',seq[-1].shape)
print('res',res.shape)

(291, 10, 1)
(21, 10, 1)
b <class 'numpy.ndarray'>
[[0.15968566]]
len 301
(291, 10, 1)
(291, 10, 1)
seq (10, 1)
res (291, 10, 1)


In [4]:
fig = plt.figure(figsize=(20, 3))  # dpi参数指定绘图对象的分辨率，即每英寸多少个像素，缺省值为80
axes = fig.add_subplot(1, 1, 1)
line3, = axes.plot(train_y[19:-1], 'r', label='实际')
plt.ion()
plt.show()
    

In [6]:
fig = plt.figure(figsize=(20, 3))  # dpi参数指定绘图对象的分辨率，即每英寸多少个像素，缺省值为80
axes = fig.add_subplot(1, 1, 1)
line4, = axes.plot(train_x[0:-1], 'b--', label='实际')
plt.ion()
plt.show()

In [14]:
def get_batch_boston_original():
    global train_x, train_y, BATCH_START, TIME_STEPS
    BATCH_START = 0
    # 每次都是拿300行，13列的数据，但是起始点根据 BATCH_START
    x_part1 = train_x[BATCH_START:BATCH_START + TIME_STEPS * BATCH_SIZE]
    y_part1 = train_y[BATCH_START:BATCH_START + TIME_STEPS * BATCH_SIZE]
    print(BATCH_START,BATCH_START + TIME_STEPS * BATCH_SIZE)
    # 时间段= 0 300
    # x_part1= (300, 13)
    # 时间段= 10 310
    # x_part1= (300, 13)
#     print('时间段=', BATCH_START, BATCH_START + TIME_STEPS * BATCH_SIZE)
#     print('x_part1=', x_part1.shape)
#     print(BATCH_SIZE, TIME_STEPS, INPUT_SIZE)
    # 转成 30 10 13的形状
    seq = x_part1.reshape((BATCH_SIZE, TIME_STEPS, INPUT_SIZE))
    res = y_part1.reshape((BATCH_SIZE, TIME_STEPS, 1))

    BATCH_START += TIME_STEPS

    # returned seq, res and xs: shape (batch, step, input)
    #np.newaxis 用来增加一个维度 变为三个维度，第三个维度将用来存上一批样本的状态
    return [seq, res]

st,rt = get_batch_boston_original()


0 300


In [ ]:
def get_batch_boston_single():
    global train_x_list, train_y_list, BATCH_START, TIME_STEPS
    train_x_list,train_y_list=[],[]
    for i in range(1):
        x1=train_y[i:i+TIME_STEPS*BATCH_SIZE]
        y1=train_y[i+1:i+TIME_STEPS*BATCH_SIZE+1]
        x1=x1.reshape(30,10,-1)
        y1=y1.reshape(30,10,-1)
        train_x_list.append(x1.tolist())
        train_y_list.append(y1.tolist()) 
    train_x_np = np.array(train_x_list)
    train_y_np = np.array(train_y_list)

    seq = train_x_np
    res = train_y_np
    return [seq, res]

In [56]:
# 训练
tf.reset_default_graph()
if __name__ == '__main__':
    # 设置画布初始属性和内容
    
#     fig = plt.figure(figsize=(20, 3))  # dpi参数指定绘图对象的分辨率，即每英寸多少个像素，缺省值为80
#     axes = fig.add_subplot(1, 1, 1)
#     line3, = axes.plot(train_y[19:-1], 'r', label='实际')
#     plt.ion()
#     plt.show()
    
    seq, res = get_batch_boston()
    costlist = []
    print('seq',seq.shape)
    print('res',res.shape)
    BATCH_SIZE  = 30
    model = LSTMRNN(TIME_STEPS, INPUT_SIZE, OUTPUT_SIZE, CELL_SIZE, BATCH_SIZE)
    sess = tf.Session()
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("logs", sess.graph)
    # tf.initialize_all_variables() no long valid from
    # 2017-03-02 if using tensorflow >= 0.12
    sess.run(tf.global_variables_initializer())
    # relocate to the local dir and run this line to view it on Chrome (http://0.0.0.0:6006/):
    # $ tensorboard --logdir='logs'
    for j in range(200):  #训练200次
        pred_res = None
        # 300
        seq, res = get_batch_boston()
#         print('seq',seq.shape)
#        用于300
        for i in range(20):
            if i == 0:
#                 print('if')
                feed_dict = {
                    model.xs: seq[i],
                    model.ys: res[i],
                    # create initial state
                }
            else:
#                 print('else')
                feed_dict = {
                    model.xs: seq[i],
                    model.ys: res[i],
                    model.cell_init_state:
                    state  # use last state as the initial state for this run
                }
#             print('se',seq[start:end].shape)
#             start += TIME_STEPS
#             print('start',type(start))
#             end=start+BATCH_SIZE

#         10
#         step=0
#         start=0
#         end=start+BATCH_SIZE
#         #每个内循环是100次，数据是0-60，60-120，120-180，递增
#         while(end<len(train_x)-206):
#             print()
#             if start == 0:
# #                 print('if')
#                 feed_dict = {
#                     model.xs: seq[start:end],
#                     model.ys: res[start:end],
#                     # create initial state
#                 }
#             else:
# #                 print('else')
#                 feed_dict = {
#                     model.xs: seq[start:end],
#                     model.ys: res[start:end],
#                     model.cell_init_state:
#                     state  # use last state as the initial state for this run
#                 }
#             print('start,end,step',start,end,step,',seq',seq[start:end].shape)
#             start+=BATCH_SIZE
#             end=start+BATCH_SIZE
#             #每10步保存一次参数
#             if step%10==0:
#                 print(i,step,cost)
#             step+=1

            
            _, cost, state, pred = sess.run([
                model.train_op, model.cost, model.cell_final_state, model.pred
            ],feed_dict=feed_dict)
            pred_res = pred
            


            result = sess.run(merged, feed_dict)
            writer.add_summary(result, i)
        if(j % 10 ==0):
            print('{0} cost: '.format(j), round(cost, 4))
        BATCH_START = 0  #从头再来一遍
#         try:
#             axes.lines.remove(line1)
#             axes.lines.remove(line_c)
#         except Exception:
#             pass
        
#         costlist.append(cost)
#         line_c, = axes.plot(costlist,'y',label='cost')
#         handles, labels = plt.gca().get_legend_handles_labels()
        # 返回是, 是一个Line2D的对象，无逗号是一个包含了Line2D的数组
#         line1, = axes.plot(
#             range(100), pred.flatten()[-100:], 'b--', label='rnn计算结果 '+str(j))
#         line1, = axes.plot(pred.flatten(), 'b--', label='rnn计算结果 '+str(j))
#         axes.grid()
#         fig.tight_layout()
#         plt.legend(handles=[line1,line3])
#         plt.title('递归神经网络')
#         plt.pause(0.01)
#     plt.ioff()
#     plt.show()
    
    # 保存模型
#     saver = tf.train.Saver()
#     saver.save(sess, save_dir,)
    print('Model Trained and Saved')

seq (20, 30, 10, 1)
res (20, 30, 10, 1)
0 cost:  5.2734
10 cost:  4.1035
20 cost:  4.0362
30 cost:  4.551
40 cost:  3.6982
50 cost:  3.7115
60 cost:  3.5777
70 cost:  3.5367
80 cost:  3.3439
90 cost:  3.0244
100 cost:  2.8598
110 cost:  2.8282
120 cost:  2.637
130 cost:  2.6433
140 cost:  2.6105
150 cost:  2.5279
160 cost:  2.519
170 cost:  2.305
180 cost:  2.3383
190 cost:  2.0244
Model Trained and Saved
